# 跳跃表简单版本

In [1]:
import random
from collections import defaultdict

![](https://raw.githubusercontent.com/luog1992/ipic/master/20211111014718.png)

In [2]:
class Node(object):
    """跳跃表中的一个节点"""
    
    def __init__(self, key, val, height):
        self.key = key
        self.val = val
        self.height = height
        # 第 i 层，当前节点指向的下一个节点
        self.nexts = [None] * height
        # 第 i 层，当前节点与 self.nexts[i] 节点间的距离
        self.spans = [0] * height

    def __repr__(self):
        return '<Node %s=%s>' % (self.key, self.val)
        

In [3]:
class SkipList(object):
    """跳跃表"""

    PROMOPT_P = 0.5   # 晋升概率
    MAX_LEVEL = 4      # 最大层高
    
    def __init__(self):
        self._len = 0
        self.head = Node(None, None, self.MAX_LEVEL)
    
    def __len__(self):
        return self._len

    def __repr__(self):
        return '<SkipList len=%s>' % self._len
    
    def _random_level(self):
        """根据晋升概率随机算出一个层高
        """
        level = 1
        while random.random() < self.PROMOPT_P:
            if level >= self.MAX_LEVEL:
                break
            else:
                level += 1
        return level
    
    def add(self, key, value, height=None):
        """添加KV
        """
        # 搜索路径上往下拐的 node
        corner_nodes = []
        # 搜索路径上，每一层经过的节点数
        search_spans = defaultdict(int)

        cur_node = self.head
        # 自上往下遍历每一层，确定搜索路径，确定要插入的位置
        for level in reversed(range(self.MAX_LEVEL)):

            # 在每一层，只要 one_node.key < key，就尽量 "往右" 移动
            while cur_node.nexts[level] and cur_node.nexts[level].key < key:
                search_spans[level] += cur_node.spans[level]
                cur_node = cur_node.nexts[level]

            # 2. 移动到不能往右移时，加 "往下"，往下拐的这个 node 是之后可能需要调整链表指针的
            corner_nodes.append((cur_node, level))
    
        # 循环结束后，cur_node 后面肯定是要插入新节点的位置
        if cur_node.nexts[0] and cur_node.nexts[0].key == key: # 0: level0
            print('node key=%s value updated from %s to %s' % (key, cur_node.nexts[0].val, value))
            cur_node.nexts[0].val = value
            return
    
        # 添加新节点
        self._len += 1
        height = height or self._random_level()
        new_node = Node(key, value, height)

        # 新节点只有 height 层，故需要调整 corner_nodes 中后 height 个节点的指针
        for the_node, level in list(reversed(corner_nodes))[:height]:
            # 修改指针
            old_next = the_node.nexts[level]
            old_span = the_node.spans[level]
            new_node.nexts[level] = old_next
            the_node.nexts[level] = new_node
            
            # 计算 the_node 和 new_node 之间的间距
            the_span = sum([s for l, s in search_spans.items() if l < level])
            the_span += 1
            the_node.spans[level] = the_span
            if old_next:
                new_node.spans[level] = (old_span + 1) - the_span
        for the_node, level in list(reversed(corner_nodes))[height:]:
            if the_node.nexts[level]:
                the_node.spans[level] += 1

    def get(self, key):
        """查找key
        """
        cur_node = self.head
        for level in reversed(range(self.MAX_LEVEL)):
            while cur_node.nexts[level] and cur_node.nexts[level].key < key:
                cur_node = cur_node.nexts[level]
    
        the_node = cur_node.nexts[level]
        if the_node and the_node.key == key:
            return the_node.val
        return None

    def graph(self):
        """画出整个跳跃表
        """
        for level in reversed(range(self.MAX_LEVEL)):
            node = self.head
            while node is not None:
                print('%s(%s)' % (node.key, node.val), end='')
                for i in range(node.spans[level] - 1):
                    print('-', end='')
                    print('%s(%s)' % ('~', '~'), end='')
                if node.spans[level] > 0:
                    print('-', end='')
                node = node.nexts[level]
            print('\n')
                

In [4]:
skip_list = SkipList()

for i in range(ord('A'), ord('Z'), 2):
    skip_list.add(chr(i), i % 10)

In [5]:
print(skip_list.get('A'))
print(skip_list.get('B'))
print(skip_list.get('Y'))

5
None
9


In [6]:
skip_list.graph()

None(None)-~(~)-~(~)-~(~)-~(~)-~(~)-K(5)

None(None)-A(5)-~(~)-~(~)-~(~)-~(~)-K(5)-~(~)-~(~)-~(~)-~(~)-~(~)-W(7)

None(None)-A(5)-C(7)-E(9)-G(1)-~(~)-K(5)-~(~)-O(9)-Q(1)-~(~)-~(~)-W(7)

None(None)-A(5)-C(7)-E(9)-G(1)-I(3)-K(5)-M(7)-O(9)-Q(1)-S(3)-U(5)-W(7)-Y(9)



In [7]:
skip_list.add('A', 55)  # modify
skip_list.get('A')

node key=A value updated from 5 to 55


55

In [8]:
skip_list.get('N')  # None
skip_list.add('N', 88, height=3)
skip_list.get('N')  # 88

88

In [9]:
skip_list.graph()

None(None)-~(~)-~(~)-~(~)-~(~)-~(~)-K(5)

None(None)-A(55)-~(~)-~(~)-~(~)-~(~)-K(5)-~(~)-N(88)-~(~)-~(~)-~(~)-~(~)-W(7)

None(None)-A(55)-C(7)-E(9)-G(1)-~(~)-K(5)-~(~)-N(88)-O(9)-Q(1)-~(~)-~(~)-W(7)

None(None)-A(55)-C(7)-E(9)-G(1)-I(3)-K(5)-M(7)-N(88)-O(9)-Q(1)-S(3)-U(5)-W(7)-Y(9)

